# Lumped Parameter Model 

Adaptado de Xiao et al (2013) Lumped parameter model for charge–discharge cycle of adsorptive
hydrogen storage system

Autor: Kieran Conradie


Las ecuaciones que gobiernan el sistema propuesto en el artículo son: 

Masa:
$$ \frac{dm_t}{dt}=\dot{m_i}-\dot{m_e}$$
Las cual puede ser separada en dos casos:
    
1. Carga: $\frac{dm_t}{dt}=\dot{m_i}$
2. Descarga: $\frac{dm_t}{dt}=-\dot{m_e}$ 

Dentro de las ecuaciones de masa existen otras que ayudan a especificar el problema: 
$$m_t = m_g + m_a$$
$$m_a = n_a M_{H_2} m_s$$

Donde los moles de hidrógeno adsorbido están dados por:
$$n_a = n_0 \exp\left[-\left(\frac{RT}{\alpha + \beta T}\right)^b \ln{\left(\frac{p_0}{p}\right)}^b\right]$$
y la presión esta dada por:
$$p=\frac{(m-m_a)ZRT}{M_{H_2}V\epsilon_b}$$

Temperatura:
$$\frac{dT}{dt}=\frac{\dot{m_i}h_i - \dot{m_e}h_e + \frac{dm_a}{dt}\frac{\Delta{H}}{M_{H_2}}-h_fA_e(T-T_f)}{m_sc_s+m_ac_{va}+m_gc_{vg}+m_wc_w}$$

Además se debe notar que en Xiao et al. (2012) Finite element simulation of heat and mass transfer in activated carbon hydrogen storage tank, se obtuvo:
$$\frac{\partial{n_a}}{\partial t}=mn_a\left(\frac{RT}{\alpha + \beta T}\ln{\left(\frac{p_0}{p}\right)}\right)^b \left[\frac{1}{\ln{(p_0/p)}}\frac{1}{p}\frac{\partial p}{\partial t}-\frac{\alpha}{\alpha + \beta T}\frac{1}{T}\frac{\partial T}{\partial t}\right]$$

In [15]:
# Integración numérica de sistemas de ecuaciones diferenciales ordinarias
from scipy.integrate import solve_ivp

# Minimización de funciones objetivos para ajuste de parámetros
from scipy.optimize import least_squares
import numpy as np
import matplotlib.pyplot as plt

### Definición de la función con el sistema de ecuaciones diferenciales

In [16]:
# Por el momento la función solo trabajara para el proceso de carga, después se le podria dar un parametro mas a la función 
# que use un kwarg del tipo proceso = carga/descarga y en base a eso realizar cálculos
def adsorcion_hidrogeno(t, y, c_s, c_p, c_w, m_s, m_w, M_H2, R, alpha, beta, epsilon, volumen, area, p_0, n_0, b, m_dot, h, h_f, T_f):
    
    # Desempacar variables
    n_a = y[0] # Isoterma de adsorcion 
    T = y[1] # Temperatura 
    p = y[2]
    
    
    # Inicializar variables como las masas (m_t, m_a, m_g) y la presión (Valor dado por las condiciones iniciales)
    m_t = 0 # Al principio del experimento el contenedor comienza vacio
    n_a = 0 # Al empezar vacio la masa adsorbida también debe ser 0, quizas se pueda usar para un tiempo "Justo" después de t = 0 s
    m_a = M_H2*m_s*n_a
    m_g = m_t - m_a # Relación dada en el artículo

    
    # Calcular cantidades dependientes de las variables independientes
    n_a = n_0 * np.exp(-((R*T)/(alpha + beta*T))**b * np.log(p_0/p)**b)
    dH = alpha * (np.log(p_0/p))**(1/b)
    
    
    # Como todas las ecuaciones dependen una de la otra se declara un valor para dn_dt y dp_dt inicial
    # Se elije resolver primero para dT_dt ya que solo depende de dn_dt y no de dp
    dp_dt = 0 # Se necesita para calcular dn_dt
    dn_dt = 0 # Se necesita para calcular dT_dt
    
    # Ecuación diferencial para la temperatura
    # Por el momento asumir que c_v = c_p para el hidrogeo (Solo por flojera, después se debe usar una correlación)
    dT_dt = (m_dot*h + M_H2*m_s*dn_dt - h_f*area*(T-T_f))/(m_s*c_s + m_a*c_p + m_g*c_p + m_w*c_w)
    
    # Ecuación diferencial para la presión
    dp_dt = R/(M_H2*epsilon*volumen) * (M_H2*m_s*dn_dt + dT_dt) 
    
    # Ecuación diferencial para la isoterma
    dn_dt = (m_t*n_a*(((R*T)/(alpha+beta*T))*np.log(p_0/p))**b * 
             ((1/np.log(p_0/p))*(1/p)*dp_dt - (alpha/(alpha+beta*T))*(1/T)*dT_dt))
    
    tol = 10e-3 # Tolerancia para encontrar la mejor aproximación para las funciones
    dif = 0 
    
    while dif > tol :
        # Número de loops realizados (Para debugear)
        n = 0
        
        # Almacenar los valores de la iteración anterior
        dT_dt_old = dT_dt 
        dp_dt_old = dp_dt
        dn_dt_old = dp_dt
    
        # Encontrar los nuevos valores de las derivadas
        dT_dt = (m_dot*H + M_H2*m_s*dn_dt - h_f*area*(T-T_f))/(m_s*c_s + m_a*c_p + m_g*c_p + m_w*c_w)
        dp_dt = R/(M_H2*epsilon*volumen) * (M_H2*m_s*dn_dt + dT_dt) 
        dn_dt = (m_t*n_a*(((R*T)/(alpha+beta*T))*np.log(p_0/p))**b * 
             ((1/np.log(p_0/p))*(1/p)*dp_dt - (alpha/(alpha+beta*T))*(1/T)*dT_dt))
        
        dif = np.abs(dT_dt - dT_dt_old) + np.abs(dp_dt - dp_dt_old) + np.abs(dn_dt - dn_dt_old)
        print(f'Iteración: {n}, la diferencia actual es: {dif}')
    
    # Empacar el vector del lado derecho en un vector 2x1
    dy = np.array([dn_dt, dT_dt, dp_dt])
    
    return dy

### Parametros y condiciones

In [18]:

# Parametros

# Calores especificos
c_s = 825/1000 # Calor especifico del carbón activado (kJ kg-1 K-1)
c_p = 10167/1000 # Calor especifico del hidrogeno (kJ kg-1 K-1)
c_w = 468/1000 # Calor especifico paredes de acero (kJ kg-1 K-1)
# Originalmente estaba dado en J, por eso el /1000

# Masas
m_s = 0.671 # masa carbón activado (kg)
m_w = 3.714 # Masa paredes de acero (kg)
M_H2 = 2.0159E-3 # Masa molar del hidregeno (kg mol-1)

# Constantes 
R = 8.314 # J mol-1 K-1
alpha = 3080 # Factor entalpico (J mol-1)
beta = 18.9 # Factor entropico (J mol-1 K-1)
epsilon_b = 0.49 
b = 2

# Dimensiones estanque
V = 2.4946 # Volumen de estanque (L)
A_e = 0.1277 # Área superficial estanque

# Otros
p_0 = 1470 # Presion de saturacion (MPa)
n_0 = 71.6 # Cantidad limite de adsorcion (mol kg-1)


# Condiciones iniciales y de borde
p_i = [0.033, 0.033, 0.033, 0.032, 0.049, 0.032] #MPa
T_i = [281, 282, 280.2, 301.5, 302, 302.4] #K 
T_f = [282.5, 284.5, 282.2, 301.7, 302.5, 302.5] #K
h_f = [36, 36, 36, 36, 36, 36]

# Seleccionar los valores del test deseado
p_i = p_i[0]
T_i = T_i[0]
T_f = T_f[0]
h_f = h_f[0]
print(f'La presiónn incial es: {p_i}\nLa temperatura incial es: {T_i}\nLa temperatura final es: {T_f}\nh_f es: {h_f}')


# Parametros en carga/descarga
# Por ahora solo se usara test No. 13
t_0 =  0            # s
t_f = 1042          # s
t_range = np.linspace(t_0, t_f, 1000)

m_dot = 2.023e-5    # kg s-1
h = 3986.8          # kJ/kg

# Tupla de parametros que se deben pasar a solve_ivp
args = (c_s, c_p, c_w, m_s, m_w, M_H2, R, alpha, beta, epsilon_b, V, A_e, p_0, n_0, b, m_dot, h, h_f, T_f)


sol = solve_ivp(adsorcion_hidrogeno, (t_0, t_f), [0, T_i, p_i], args=args, t_eval= t_range)

La presiónn incial es: 0.033
La temperatura incial es: 281
La temperatura final es: 282.5
h_f es: 36


C:\Users\kiera\AppData\Local\Temp\ipykernel_6140\1442870420.py:20: RuntimeWarning: invalid value encountered in double_scalars
  dH = alpha * (np.log(p_0/p))**(1/b)


In [25]:
sol.y[2]

array([3.30000000e-02, 4.48921985e+03, 5.04130268e+03, 5.11069684e+03,
       5.11895390e+03, 5.11888427e+03, 5.12235430e+03, 5.11804981e+03,
       5.12091278e+03, 5.12271754e+03, 5.11439866e+03, 5.12611429e+03,
       5.11345881e+03, 5.11824271e+03, 5.12196272e+03, 5.11765121e+03,
       5.11939258e+03, 5.12408281e+03, 5.11365540e+03, 5.12622510e+03,
       5.11521199e+03, 5.11675036e+03, 5.12211936e+03, 5.11789513e+03,
       5.11821730e+03, 5.12371001e+03, 5.11542489e+03, 5.12396876e+03,
       5.11917031e+03, 5.11442799e+03, 5.12285210e+03, 5.11751152e+03,
       5.11736376e+03, 5.12265365e+03, 5.11850309e+03, 5.12053993e+03,
       5.12111780e+03, 5.11471336e+03, 5.12118738e+03, 5.12151627e+03,
       5.11593814e+03, 5.12550955e+03, 5.11456451e+03, 5.12062157e+03,
       5.12177157e+03, 5.11354976e+03, 5.12035236e+03, 5.12182570e+03,
       5.11601898e+03, 5.12360139e+03, 5.11887086e+03, 5.11781967e+03,
       5.12410120e+03, 5.11225244e+03, 5.11964649e+03, 5.12171052e+03,
      